# Model Experimentation
This notebook is designed to streamline experimentation with different models.

1. Selecting a model, dataset, tracker and hyperparameters
2. Training the model and evaluating both object detection and tracking performance
3. Saving the results in wandb
4. Storing the model and evaluation annotations in a folder
5. **Investigation** of the results, detection of failure cases and model improvement
6. **Iterating** over steps 1-5



In [1]:
from architectures import yolov8
from trackers import botsort
import wandb
import json

> run `wandb login` in terminal before running this notebook

### Select a model, dataset, tracker and hyperparameters

In [2]:
architectures = {
  "yolov8": yolov8.YoloV8
  }
trackers = {
  'botsort': botsort.BotSort
}
dataset_mapping = {} # map dataset name to dataset class

In [3]:
required_train_params = ["architecture", "data", "epochs", "batch_size", "img_size", "lr", "greyscale", "model_size", "model_path"]
required_eval_params = ["conf_treshold", "eval_data", "iou_association_threshold", "tracker"]
dynamic = ["pretrained", "annotation_path"]

def construct_hyperparameters(model_name, **kwargs):
  """
  - train params: architecture, path, architecture, data, epochs, batch_size, img_size, train_mins, device
  - eval params: conf_tresh, device, eval_data
  """
  hyperparameters = {
    "model_name": model_name
  }

  model_pretrained = False

  with open("./assets/trained_models.json", "r") as file:
      trained_models = json.load(file)
  assert trained_models is not None
  
  if model_name in trained_models:
     model_pretrained = True
     # Train params are already known  
     model_train_params = trained_models[model_name]
     assert model_train_params is not None and [param in model_train_params for param in required_train_params]
     hyperparameters.update({param: model_train_params[param] for param in required_train_params})
     hyperparameters.update({param: kwargs[param] for param in required_eval_params})

  else:
    # TODO: account for tf models
    # TODO: allow for train_min, device s to be added afterwards
    model_path = "/vol/biomedic3/bglocker/ugproj2324/fv220/models/best.pt"
    assert len(kwargs) == len(required_train_params) + len(required_eval_params)
    assert [param in kwargs for param in required_train_params + required_eval_params]
    hyperparameters.update({**kwargs, "path": model_path})

  hyperparameters['pretrained'] = model_pretrained
  hyperparameters['annotations_path'] = "/".join(hyperparameters['model_path'].split("/")[:-1]) + "/annotations.csv"

  assert [param in hyperparameters for param in required_train_params + required_eval_params + dynamic]

  return hyperparameters

In [4]:
def model_experimentation(hyperparameters):
  # run = wandb.init(project="SharkTrack", config=hyperparameters)

  try:
    # dataset_name = hyperparameters['dataset_name']

    tracker = trackers[hyperparameters['tracker']]()
    model = architectures[hyperparameters['architecture']](hyperparameters, tracker)
    # dataset = dataset_mapping[dataset_name]

    # # model.fit(dataset)
    mota, motp, idf1 =  model.evaluate()

    # wandb.log({"MOTA": mota, "MOTP": motp, "IDF1": idf1})
    # TODO: add image as well, test+time and test_device

    # Update trained_models[hyperparameters[model_name]] with new hyperparameters
    with open("./assets/trained_models.json", "r") as file:
        trained_models = json.load(file)
        assert trained_models is not None
        trained_models[hyperparameters['model_name']] = hyperparameters
        trained_models['annotation_path'] = hyperparameters['annotations_path']
        with open("./assets/trained_models.json", "w") as file:
          json.dump(trained_models, file)
     
  finally: 
    pass
    # run.finish()
  


In [5]:
model_name = "yoloV8-medium-mvd2"
eval_params = {
  "conf_treshold": 0.2,
  "eval_data": "eval1",
  "iou_association_threshold": 0.5,
  "tracker": "botsort"
}

hyperparameters = construct_hyperparameters(model_name, **eval_params)
print(hyperparameters)

model_experimentation(hyperparameters)

{'model_name': 'yoloV8-medium-mvd2', 'architecture': 'yolov8', 'data': 'MVDv2', 'epochs': 50, 'batch_size': 16, 'img_size': 640, 'lr': 0.01, 'greyscale': True, 'model_size': 'm', 'model_path': '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/best.pt', 'conf_treshold': 0.2, 'eval_data': 'eval1', 'iou_association_threshold': 0.5, 'tracker': 'botsort', 'pretrained': True, 'annotations_path': '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/annotations.csv'}


FileNotFoundError: [Errno 2] No such file or directory: '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/best.pt'